In [220]:
import data
import torch
from torch.autograd import Variable
import pickle
import random
from skll.metrics import kappa
import numpy as np

In [211]:
with open('word_to_ix.pk', 'rb') as handle:
    word_to_ix = pickle.load(handle)
print("Finished Loading Workd to Index Dictionary")
with open('test_set.pk', 'rb') as handle:
    test_set = pickle.load(handle)

Finished Loading Workd to Index Dictionary


In [212]:
model1 = torch.load('lstm-mot-emb3.model', map_location=lambda storage, loc: storage)
model1

/anaconda3/lib/python3.6/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'Model.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


LSTM (
  (encoder): Embedding(12128, 50)
  (rnn): LSTM(50, 24, bias=False, dropout=0.5, bidirectional=True)
  (decoder): Linear (24 -> 1)
  (decoder_bi): Linear (48 -> 1)
)

In [238]:
PADDING = "<PAD>"
UNKNOWN = "<UNK>"
def sample_padded_index_sequences(word_indices, example,score, max_seq_length):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    temp = {}
    temp['text'] = example
    temp['label'] = score
    example = temp
    example['text_index_sequence'] = torch.zeros(max_seq_length)

    token_sequence = data.tokenize(example['text'])
    padding = max_seq_length - len(token_sequence)

    for i in range(max_seq_length):
        if i >= len(token_sequence):
            index = word_indices[PADDING]
            pass
        else:
            if token_sequence[i] in word_indices:
                index = word_indices[token_sequence[i]]
            else:
                index = word_indices[UNKNOWN]
        example['text_index_sequence'][i] = index

    example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
    example['label'] = torch.FloatTensor([example['label']])
    return example

def sentences_to_padded_index_sequences(word_indices, dataset, max_seq_length):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    for j, example in enumerate(dataset):
        example['text_index_sequence'] = torch.zeros(max_seq_length)

        token_sequence = data.tokenize(example['text'])
        padding = max_seq_length - len(token_sequence)

        for i in range(max_seq_length):
            if i >= len(token_sequence):
                index = word_indices[PADDING]
                pass
            else:
                if token_sequence[i] in word_indices:
                    index = word_indices[token_sequence[i]]
                else:
                    index = word_indices[UNKNOWN]
            example['text_index_sequence'][i] = index

        example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
        example['label'] = torch.LongTensor([example['label']])


def evaluate_sample(model, converted, batch_size=1):
    model.eval()
    hidden = model.init_hidden(batch_size)
    vectors = converted["text_index_sequence"].view(-1,1)
    labels = converted["label"].type('torch.FloatTensor')
    vectors, labels = Variable(vectors), Variable(labels)
    output, hidden = model(vectors, hidden)
    predicted = output.data.cpu().numpy()
    print('predicted: %d, true label: %d' %(predicted, labels.data.cpu().numpy()))

def evaluate_kappa(model, data_iter):
    scale_map = {1:1, 2:4/5, 3:3/10, 4:3/10, 5:1/2.5, 6:1/2.5, 7:3, 8: 6}
    bias_map = {1:+2, 2:+2, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0}
    
    model.eval()
    predicted_labels = []
    true_labels = []
    
    hidden = model.init_hidden(len(data_iter))

    vectors, labels, essay_set = get_batch(data_iter)
    vectors = vectors.transpose(1, 0)
    vectors = Variable(vectors)

    output, hidden = model(vectors, hidden)
    
    #return output, hidden
    predicted = [float(num) for num in output.data.cpu().numpy()]
    de_scale = [(p*scale_map[i])+bias_map[i] for p,i in zip(predicted,essay_set)]

    return kappa(labels, de_scale, weights = "quadratic")


        
# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    essay_set = []
    for dict in batch:
        vectors.append(dict["text_index_sequence"])
        labels.append(dict["label"])
        essay_set.append(dict["essay_set"])
        
    vectors = torch.stack(vectors).squeeze()
    labels = torch.stack(labels).squeeze().cpu().numpy()
    #labels = Variable(torch.stack(labels).squeeze().type('torch.FloatTensor'))
    return vectors, labels, essay_set


def main(sample, score):
    print("Essay length: %s words."%len(data.tokenize(sample)))
    print("-"*50)
    converted = sample_padded_index_sequences(word_to_ix, sample,score, max_seq_length=1064)
    text_index = converted['text_index_sequence']
    print('Converted Text: %s' % text_index)
    print("-"*50)
    print('Converted Embeddings: %s' % model1.encoder(Variable(text_index.view(-1))))
    print("-"*50)
    evaluate_sample(model1, converted, batch_size=1)

In [214]:
sentences_to_padded_index_sequences(word_to_ix, test_set, max_seq_length = 1064)

In [229]:
p, es,de_scale, labels = evaluate_kappa(model1, test_set)

In [235]:
es.index(2)

7

In [224]:
labels

array([12, 12,  0, ...,  8,  9,  6])

### Topic1 sample:

In [167]:
sample1 = 'for some people it is as hard as a rock, but to some it just comes naturally. patience is a big part of hunting. you must be tolerant and understanding. even if things seem to take a turn for the worst. take this for example. i had woke up at six and jumped out of bed. i was finally going to shoot my first deer! i pulled on my camofage and hunter’s orange as my grandpa drove down my long drive way to get me. the moon light shone bright and i was giggling with exitment, but i knew i had to calm down. as we drove to the place i was going to hunt i was amazed by how quite evreything was. we walked back to the blind on a leaf covered trail and set evrething in it’s place. after, dosing of a little the morning light seemed to peek through the trees and light up the fresh green field. the water vapor looked like sparkles floting in mid air. we heard a grouse drum and had a red squarle jump out at us, but no deer. after a long morning of sitting we disided to pack up. we always had tomarrow. so that was the end of our hunt. we had an exiting time, and it was all because of patience. patience @month1 not always help in ways you wish, but it can help you see many diffrent and beautiful things.'
score1 = 6

In [168]:
main(sample1,score1)

Essay length: 272 words.
--------------------------------------------------
Converted Text: 
  9044   2331   3240  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 1.4352e+00 -4.2654e-01  1.2005e+00  ...  -4.4088e-01  2.8095e-02 -6.8341e-01
 4.9672e-01 -5.0910e-01 -3.8796e-01  ...   2.1977e-01 -1.1339e-01  8.0408e-01
 3.8238e-01 -2.0466e-01  2.1652e-01  ...  -9.4410e-02 -2.7204e-01 -2.7673e-01
                ...                   ⋱                   ...                
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 21, true label: 6


### Topic2 sample:

In [169]:
sample2 = 'Write a persuasive essay to a newspaper reflecting your views on censorship in libraries. Do you believe that certain should be removed i think so be no that yes i think should no person that in chager the book, music, movies, magazines, ect., that be no agure      why do i think if you need that please  think i no thank you please if  i need why do we if know that if i failure the this test i who need to graduate please the children allow to home please yes.          Why do we need to be a prafece person please why do we need to do this why write this assgiment because you mean to be the best teaches ever and ever facebook is my password is @PERSON1  @NUM1 that why i need my myspace is the same thingh but different at same time please know that i need to know i really  i need to my e-mail address is  @EMAIL1 that is my e-mail please work m'
score2 = 3

In [170]:
main(sample2,score2)

Essay length: 175 words.
--------------------------------------------------
Converted Text: 
     1   7771   4917  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 0.5411 -0.2377  0.1376  ...  -0.1364 -0.2760 -0.1484
 0.3445 -0.2953  0.2469  ...  -0.0460 -0.3059 -0.3119
-0.0269  0.1645 -0.1006  ...   0.0333  0.1535  0.1081
          ...             ⋱             ...          
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 3, true label: 3


### Topic3 sample:

In [490]:
sample3 = 'Everyone travels to unfamiliar places. Sometimes we get lost and ask locals for directions which @MONTH1 not be a good idea. The setting affected the cyclist. He had a perfectly good map but asked older men who look like they haven’t been out in ages. The old men apparently haven’t been out because they gave the cyclist the wrong directions.  My advice would be to not listen to anyone no matter their age if you have a perfectly good map with you. Also try to know where you are going at all times. “Yes, sir!'
score3 = 1

In [491]:
main(sample3,score3)

Essay length: 95 words.
--------------------------------------------------
Converted Text: 
     1  10528   6631  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 5.4112e-01 -2.3767e-01  1.3759e-01  ...  -1.3637e-01 -2.7599e-01 -1.4840e-01
-4.1228e-01  1.1046e+00 -2.4505e+00  ...   6.2484e-01 -1.8060e-01  3.1754e-01
 5.6562e-01 -2.2704e-01  1.3931e-01  ...  -1.0363e-01 -3.8096e-01 -1.3700e-01
                ...                   ⋱                   ...                
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 0, true label: 1


### Topic5 sample:

In [492]:
sample5 = "In this memoir of Narciso Rodriguez, @PERSON3's life, the mood containing it all, was greatful, and showed how his parents gave him love through his whole life. Both his parents were born and raised in Cuba, and in 1956, they both moved to the United States. Starting their lives over again, and taking any job they could find. Then in 1961, @PERSON2, @CAPS1. was born. Both Parents raised him with love, and care, and introducing his Cuban background into his life.        As @PERSON2, was telling his story, all readers could tell how greatful he was to have a family like he did. Also, as he shared his non-– blood related family, and the remembrence on how his parents' life changed by moving to @LOCATION2, he showed the respect he had towards them."
score5 = 2

In [493]:
main(sample5,score5)

Essay length: 132 words.
--------------------------------------------------
Converted Text: 
     1   6957  10407  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 0.5411 -0.2377  0.1376  ...  -0.1364 -0.2760 -0.1484
 0.3527 -0.0782  0.0604  ...   0.1353 -0.2465  0.0095
-0.4991  1.2075 -1.2546  ...  -0.2768  0.4714 -0.1080
          ...             ⋱             ...          
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 1, true label: 2


### Topic6 sample:

In [494]:
sample6 = 'There was a girl name @PERSON1. She loved spending time with her mom. Every weekend they would either @CAPS3 the movies, amusement or water park, or just stay home. This time was different though. @PERSON1 was even a little scared about @CAPS1 was about @CAPS3 happen. Today was a @DATE1 and @PERSON1 and her mom was watching videos of her mom when she was young. After the video, @PERSON1 and her mom looked at each other and laughed. Once they settled down, @PERSON1’s mom looked sad. “@CAPS1’s wrong mom?” @PERSON1 asked. “I have @CAPS3 go. I have @CAPS3 go somewhere for work and can’t take you with me,” mom replied. “@CAPS2 not?” asked @PERSON1. “I don’t know, but I need you @CAPS3 wait here with your father. I’ll be back soon,” the mom said calmly. With that @PERSON1’s mom left and drove away. @PERSON1 stood there confused and wishing her mom didn’t have @CAPS3 go. Then she went @CAPS3 the phone and called her dad. An hour later, @PERSON1’s dad came @CAPS3 her house. He calmed @PERSON1 down, who had been crying because she missed her mom. @PERSON1 went @CAPS3 her room and began  @CAPS3 wait patiently for her mom. Weeks past and @PERSON1’s mom still hasn’t returned. Every night now @PERSON1 cried hoping her mom would come. During the day she would wait watching for her mom. @PERSON1 wanted her mom more than ever now. Though see never complained @CAPS3 her father and understands that her mom left for an important reason, but still she missed her mom. Two years have passed since @PERSON1’s mom had left. @PERSON1 didn’t cry as much but still waited. Waited for her mother @CAPS3 come back. One day @PERSON1 heard a knock on the door, she answered it and there stood her mother. They stood on the porch for a moment, then hugged each other. They both cried and her mom whispered “I’m sorry.” @CAPS3 @PERSON1. Now @PERSON1 and her were unseperatable. They never left each other’s side again.'
score6 = 23

In [495]:
main(sample6,score6)

Essay length: 336 words.
--------------------------------------------------
Converted Text: 
     1   8523   7771  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 0.5411 -0.2377  0.1376  ...  -0.1364 -0.2760 -0.1484
 0.3281 -0.8774 -0.9861  ...   0.2076 -0.1580  0.1662
 0.3445 -0.2953  0.2469  ...  -0.0460 -0.3059 -0.3119
          ...             ⋱             ...          
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 19, true label: 23


### Topic7 sample:

In [500]:
sample7 = "There was a girl name @PERSON1. She loved spending time with her mom. Every weekend they would either @CAPS3 the movies, amusement or water park, or just stay home. This time was different though. @PERSON1 was even a little scared about @CAPS1 was about @CAPS3 happen. Today was a @DATE1 and @PERSON1 and her mom was watching videos of her mom when she was young. After the video, @PERSON1 and her mom looked at each other and laughed. Once they settled down, @PERSON1’s mom looked sad. “@CAPS1’s wrong mom?” @PERSON1 asked. “I have @CAPS3 go. I have @CAPS3 go somewhere for work and can’t take you with me,” mom replied. “@CAPS2 not?” asked @PERSON1. “I don’t know, but I need you @CAPS3 wait here with your father. I’ll be back soon,” the mom said calmly. With that @PERSON1’s mom left and drove away. @PERSON1 stood there confused and wishing her mom didn’t have @CAPS3 go. Then she went @CAPS3 the phone and called her dad. An hour later, @PERSON1’s dad came @CAPS3 her house. He calmed @PERSON1 down, who had been crying because she missed her mom. @PERSON1 went @CAPS3 her room and began  @CAPS3 wait patiently for her mom. Weeks past and @PERSON1’s mom still hasn’t returned. Every night now @PERSON1 cried hoping her mom would come. During the day she would wait watching for her mom. @PERSON1 wanted her mom more than ever now. Though see never complained @CAPS3 her father and understands that her mom left for an important reason, but still she missed her mom. Two years have passed since @PERSON1’s mom had left. @PERSON1 didn’t cry as much but still waited. Waited for her mother @CAPS3 come back. One day @PERSON1 heard a knock on the door, she answered it and there stood her mother. They stood on the porch for a moment, then hugged each other. They both cried and her mom whispered “I’m sorry.” @CAPS3 @PERSON1. Now @PERSON1 and her were unseperatable. They never left each other’s side again."
score7 = 23

In [501]:
main(sample7,score7)

Essay length: 336 words.
--------------------------------------------------
Converted Text: 
     1   8523   7771  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 0.5411 -0.2377  0.1376  ...  -0.1364 -0.2760 -0.1484
 0.3281 -0.8774 -0.9861  ...   0.2076 -0.1580  0.1662
 0.3445 -0.2953  0.2469  ...  -0.0460 -0.3059 -0.3119
          ...             ⋱             ...          
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
 0.0097  0.1926  0.0754  ...  -0.0256 -0.2809 -0.0458
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 19, true label: 23


### Topic8 sample:

In [504]:
sample8 = " I believe that with all people laughter, and having a sense of humor, is something that generally everyone has in common, everyone loves to share with others. Among young teenage girls, laughter is a commodity that is infectious but always wonderful. It's the purest kind of joy we have, it's somethings that brings humans together. It's the beginning and end of friendship and unity. Laughter can ring out and lift the spirits of those that really need it. A simple laugh can make the worst day a little alright. We all know that feeling of dread, and sadness . The kindness, generosity, and good of laughter is one of the few things that are right nowadays. Even in poverty, depression, and oppression the simple joy of laughter can wash away a person's worries for a split second and fill us with utter warmth.  @PERSON1 and I have been the best of friends since we were ten-years-old, and laughter has always been our strongest element. @PERSON1 and I have a knack for making each other burst out in laughter when all we want to do is be angry or just cry. We've decided that apart, we're just a little funny, but with our powers combined we could make anyone laugh. We use that power on each other when we're having bad days, or we simply need some cheering up. I think if it wasn't for laughter we wouldn't even be friends anymore. It really is what brings us together as best friends. We'd be different people. Even if I drifted from her we'd always be cracking jokes, making each others guts hurt, and tearing up from laughing so hard.  My first impression of @PERSON1 was of her stomping down to our neighborhood park to yell at her sister, whom I had been playing with at the time. It would be a very self-explanatory reason of why I didn't like her at first. Luckily, after going to @PERSON1's house with her sister, @PERSON1 and I found that we had many things in common, and we were only four months apart. That childish kind of cajoling held our friendship together, for it was a bit rocky at the beginning. Always fighting, and arguing about things that now, we look back on and laugh at our adolescent stupidity. Either one of us would be popping off with dumb little comments at each other when neither of us wanted to apologize. One time she proceeded to tell me that I couldn't hear her because I had a big bug in my ear. Now we can remind each other of things like that and have a good ole time laughing about it. We were mere @NUM1-year-old's then. Now, almost @NUM2, we rarely fight, but we always laugh and make the best of what we've got with our poking fun. Laughter was definitely the band-aid in the beginning of our rickety friendship. @PERSON1 has a gift to make anything funny at any time. She says the wrong (but hilarious) things, but always at the right time. Laughter is now a necessity between us, it's what keeps our friendship so strong. I think of what the human race would be like if laughter was not a factor in everyday life. Everyone knows that it wasn't a good day if you haven't laughed or let alone smiled. People would be dull, and just unhappy. Laughter heals people from sickness, it gives us the feeling that maybe everything is going to be okay after all. I truly believe that it's healthy for you. With a flip of a switch, laughing can bring within us an ecstatic, and serendipitous feeling. People take it for granted and use it for wrong when it's meant to spread the word of happiness. We're lucky to have a gift of such natural joy. Can anyone even explain, in detail, the feeling laughter truly brings us? I think not. It's like describing the scent of something. You just can't do it. The words aren't there. I think of those poor and miserable people that have forgotten what it feels like to laugh, and it's very sad. I strive to laugh every single day, and make at least one other person laugh. What is a life without a diamond in the rough like laughter? It's no life at all, for laughter will always be your shelter on the pursuit of happiness."
score8 = 40

In [505]:
main(sample8,score8)

Essay length: 734 words.
--------------------------------------------------
Converted Text: 
     1   8764   4878  ...       0      0      0
[torch.LongTensor of size 1x1064]

--------------------------------------------------
Converted Embeddings: Variable containing:
 5.4112e-01 -2.3767e-01  1.3759e-01  ...  -1.3637e-01 -2.7599e-01 -1.4840e-01
 2.3633e-01 -2.6898e-01  1.8536e-01  ...  -9.8821e-01 -5.1257e-01  4.4771e-01
 4.6780e-01  4.0498e-02  4.1466e-01  ...  -1.2841e+00 -3.8496e-01 -1.1605e+00
                ...                   ⋱                   ...                
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
 9.7310e-03  1.9256e-01  7.5418e-02  ...  -2.5582e-02 -2.8090e-01 -4.5839e-02
[torch.FloatTensor of size 1064x50]

--------------------------------------------------
predicted: 34, true label: 40
